### Format Knowledge-Graph Embeddings for Hopwise `dataset.get_preload_weight()` function
This notebook shows you how you can format kge methods embeddings to be loaded with `dataset.get_preload_weight`


📚 [Load Pretrained Embedding Documentation](https://recbole.io/docs/user_guide/usage/load_pretrained_embedding.html)

**Load Libraries**

In [1]:
import torch
import os
import pandas as pd
import numpy as np
import torch.nn as nn
from hopwise.data import create_dataset

### Load Checkpoint

In [2]:
checkpoint_name = 'saved/TransE-Feb-25-2025_12-22-01.pth'

In [3]:
checkpoint = torch.load(checkpoint_name)

/tmp/ipykernel_4178078/510420941.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_name)
/home/asoccol/hopwise/.venv/lib/python3.9/site-

**The Embeddings detected are**

In [4]:
checkpoint['state_dict'].keys()

odict_keys(['user_embedding.weight', 'entity_embedding.weight', 'relation_embedding.weight'])

**Do you want to exclude some embeddings?**

In [5]:
excluded = ['relation_bias_embedding.weight']

**The Dataset detected is**

In [6]:
dataset_name = checkpoint['config']['dataset']
dataset_name

'ml-100k'

**The Dataset folder detected is**

In [7]:
data_path = checkpoint['config']['data_path']
data_path

'/home/asoccol/hopwise/hopwise/config/../dataset_example/ml-100k'

**Create the mappings between embedding and original entity/relation/user**

- Users have a mapping 1-1 so we don't need a mapping.

- We suppose that indexing starts at 1. (tipically 0 is reserved for [PAD])

In [8]:
dataset = create_dataset(checkpoint['config'])

In [9]:
# create the reverse mapping
eid2token = {id: token for token, id in dataset.field2token_id['head_id'].items()}
rid2token = {id: token for token, id in dataset.field2token_id['relation_id'].items()}
print(rid2token)

{0: '[PAD]', 1: 'film.producer.film', 2: 'film.film.actor', 3: 'film.film_genre.films_in_this_genre', 4: 'film.film.genre', 5: 'film.writer.film', 6: 'film.film.award_nomination', 7: 'film.film.directed_by', 8: 'film.cinematographer.film', 9: 'film.film.produced_by', 10: 'film.film.subjects', 11: 'film.content_rating.film', 12: 'film.production_company.films', 13: 'film.film.written_by', 14: 'film.film.country', 15: 'film.film.language', 16: 'film.film_subject.films', 17: 'film.film.rating', 18: 'film.film.production_companies', 19: 'film.film.award_won', 20: 'film.director.film', 21: 'film.film.cinematography', 22: 'film.film.sequel', 23: 'film.film.prequel', 24: 'film.actor.film', 25: '[UI-Relation]'}


In [10]:
# # add dummy relation, check kge code
# rid2token[len(rid2token)] = 'ui_dummy_relation'

In [11]:
assert (len(eid2token.keys()) == checkpoint['state_dict']['entity_embedding.weight'].shape[0])
assert (len(rid2token.keys()) == checkpoint['state_dict']['relation_embedding.weight'].shape[0])

*if the assertion check fails, make sure that you've trained the kge without adding dummy relations/entities explicitly when creating relation/entity embeddings!*

### Create the new embeddings

In [12]:
def format_embedding(weight, columns, emb_type):
    weight = weight.detach().cpu().numpy()
    new_emb_dict = {columns[0]: list(), 
                    columns[1]: list() }
    
    if emb_type == 'entity':
        mapping = eid2token    
    elif emb_type == 'relation':
        mapping = rid2token
    else:
        mapping = None
    # Create index
    new_emb_dict[columns[0]] = [mapping[id] if mapping is not None else id for id in range(1,len(weight))]

    # Create embedding
    new_emb_dict[columns[1]] = [" ".join(f"{x}" for x in row) for row in weight[1:]]
    
    filename = f'{dataset_name}.{emb_type}emb'
    df = pd.DataFrame(new_emb_dict)
    print(f"[+] Saving the new {dataset_name} {columns[0]} embedding in {data_path}/{filename}!")
    df.to_csv(os.path.join(data_path,filename), sep='\t',index=False)

In [13]:
for emb_name, emb in checkpoint['state_dict'].items():
    if emb_name in excluded:
        continue
    # What is? Entity? User? Relation? Item? 
    emb_type = emb_name.split("_")[0]
    # Create the new embedding file columns
    columns = [f'{emb_type}id:token', f'{emb_type}_embedding:float_seq']
    print(f"[+] Formatting {emb_name} with columns {columns}")
    format_embedding(emb, columns, emb_type)

[+] Formatting user_embedding.weight with columns ['userid:token', 'user_embedding:float_seq']
[+] Saving the new ml-100k userid:token embedding in /home/asoccol/hopwise/hopwise/config/../dataset_example/ml-100k/ml-100k.useremb!
[+] Formatting entity_embedding.weight with columns ['entityid:token', 'entity_embedding:float_seq']
[+] Saving the new ml-100k entityid:token embedding in /home/asoccol/hopwise/hopwise/config/../dataset_example/ml-100k/ml-100k.entityemb!
[+] Formatting relation_embedding.weight with columns ['relationid:token', 'relation_embedding:float_seq']
[+] Saving the new ml-100k relationid:token embedding in /home/asoccol/hopwise/hopwise/config/../dataset_example/ml-100k/ml-100k.relationemb!


### Next?

Now, in the dataset folder there are these file

In [14]:
os.listdir(data_path)

['ml-100k.user',
 'ml-100k.relationemb',
 'ml-100k.item',
 'ml-100k.inter',
 'ml-100k.useremb',
 'ml-100k.link',
 'ml-100k.entityemb',
 'ml-100k.kg']

**We want to make sure that the dataset configuration is ok.**

Suppose that the output of the format embedding phase is:

```text
    [+] Formatting user_embedding.weight with columns ['userid:token', 'user_embedding:float_seq']
    [+] Saving the new ml-1m userid:token embedding in /home/recsysdatasets/ml-1m/ml-1m.useremb!
    [+] Formatting entity_embedding.weight with columns ['entityid:token', 'entity_embedding:float_seq']
    [+] Saving the new ml-1m entityid:token embedding in /home/recsysdatasets/ml-1m/ml-1m.entityemb!
    [+] Formatting relation_embedding.weight with columns ['relationid:token', 'relation_embedding:float_seq']
    [+] Saving the new ml-1m relationid:token embedding in /home/recsysdatasets/ml-1m/ml-1m.relationemb!
```

Then, you should go to the dataset configuration file (in our case is in `hopwise/properties/dataset/ml-1m.yaml`) and add the new files to be loaded


```text
    additional_feat_suffix: [useremb, entityemb, relationemb]  
    load_col:                                                  
        useremb: [userid, user_embedding]
        entityemb: [entityid, entity_embedding]
        relationemb: [relationid, relation_embedding]
    
    alias_of_user_id: [userid]
    alias_of_entity_id: [entityid]
    alias_of_relation_id: [relationid]
    
    preload_weight:
      userid: user_embedding
      entityid: entity_embedding
      relationid: relation_embedding

```



### The end

Now in your code you should be able to access to pretrained embeddings in your model through:

*Torch*
```python
    pretrained_user_emb = dataset.get_preload_weight('userid')
    pretrained_entity_emb = dataset.get_preload_weight('entityid')
    pretrained_relation_emb = dataset.get_preload_weight('relationid')
    
    self.user_embedding = nn.Embedding.from_pretrained(torch.from_numpy(pretrained_user_emb))
    self.entity_embedding = nn.Embedding.from_pretrained(torch.from_numpy(pretrained_entity_emb))
    self.relation_embedding = nn.Embedding.from_pretrained(torch.from_numpy(pretrained_relation_emb))
```

*Numpy*:
```python
    self.pretrained_user_emb = dataset.get_preload_weight('userid')
    self.entity_embedding = dataset.get_preload_weight('entityid')
    self.relation_embedding = dataset.get_preload_weight('relationid')
```


